# Entry 2: Bitmap (BMP)
## Take a good look

As WAV was one of the simplest ways to hold an audio file, bitmap is one of the simplest ways to hold an image file. Like WAV, there's no compression, lossy or otherwise.

## Context: What is stored?

If you have sight, it's probably one of your keenest senses. Eyes tell us all about our environment. It was natural that computers would go beyond text to show images.

Our eyes and cameras see light as it hits a plane. We have no real idea where the photons are going, just what their wavelength are and where they hit. So each of our eyes and a single camera will see a plane of wavelengths.

While our audio files map out a single signal over one dimension, times the number of channels, images are a two dimensional signal best mapped on a grid. Further, there's the question of storing color in each cell of this grid. And in a sense, there's another level of depth in terms of the number of bits that go into describing the color in each cell.

Color space is its own issue. While we can imagine most colors as a mixture of red, green, and blue light, and easily store color as three channels of color, we do not percieve all colors equally. Further, in low light, we percieve less color, and in near-darkness, we barely percieve color at all, because our retina's monochromatic rod cells function in low light unlike our color-sensitive cone cells.

Here is a chart of where are eyes are most and least sensitive to color, assuming you aren't colorblind. [Courtesy of Wikimedia](https://en.wikipedia.org/wiki/File:Eyesensitivity.svg).

![img](https://i.imgur.com/WF1fIJj.png)

As you can see, our eyes are most attuned to greens, and less to more violet and red shades. From an evolutionary perspective, it's speculated this is to do with the importance of distinguishing plants in a prehistoric environment. But this is a blogpost about image formats and not how we evolved to be the smartest apes.

But the past two paragraphs aren't extremely relevant here. But imagine how this knowledge could make a more compact/effective image format.

Rather, what the BMP bitmap format does is store a list of RGB values, and assigns that index to each color. This means a single integer stands in for what would be a three integer structure. It reminds me a bit of the [palette systems older video game consoles used](https://en.wikipedia.org/wiki/List_of_video_game_console_palettes).

## The Metadata

For this example, I would like to use a bitmap conversion of our tapir friends from Entry 0. The version you're seeing below is a JPG, because bitmaps actually take up a good amount of space. It'd be a waste of bandwidth. How is it so much smaller? That's another post.

![tapir](https://i.imgur.com/Eb3rHmNl.jpg)

In [6]:
# console time
# converting our image from jpg to bmp using ImageMagick, a free and useful command line image utility
! convert tapir.jpg tapir.bmp
! du -h tapir.jpg
! du -h tapir.bmp
! file tapir.bmp

512K	tapir.jpg
5.1M	tapir.bmp
tapir.bmp: PC bitmap, Windows 98/2000 and newer format, 1535 x 1150 x 24


Now we have a new tapir bitmap! Nice! And we know ImageMagick did its job because we double checked with `file`.

BMP is indeed a Windows format, and the version we're looking at is pretty old. I'm consulting [these specifications of the format](https://msdn.microsoft.com/en-us/library/windows/desktop/dd183391(v=vs.85).aspx) from Microsoft's website.

Let's open this stuff up in python and start reading some structs.

In [9]:
import struct

# read our file as a binary
with open("tapir.bmp", "rb") as f:
    bmp = f.read()
    
img_obj = {}



Like in our last file, we have a string of bytes. Let's make out our headers. Per the Microsoft documentation, we have:

![img](https://i.imgur.com/WRHSH6l.png)

Let's first pull out that BITMAPFILEHEADER.

That looks like:

![http://i.imgur.com/uJmSbb5.png](http://i.imgur.com/uJmSbb5.png)

Here, a WORD is a 16 bit unsigned int, and a DWORD is a 32 bit unsigned int. So those are our field sizes. bfType is actually two chars, `BM`, who are our magic number. bfSize is an integer of the file size. Both reserved fields are empty, presumably they were added as future-proofing so the format can be expanded. bfOffBits is the number of bytes to the first bits in the actual bitmap.

Let's unpack those fields from the image.

In [18]:
# header length is 2 + 4 + 2 + 2 + 4 bytes, or 14 bytes
# Interesting fact, you need an = prefix to ignore alignments
bfType, bfSize, bfReserved1, bfReserved2, bfOffBits = struct.unpack("=2sIHHI", bmp[:14])

print("This is a {} image, that's {} bytes long, and those bits for the bitmap are just {} bytes from index 0."
     .format(bfType.decode("UTF-8"), bfSize, bfOffBits))

This is a BM image, that's 5299338 bytes long, and those bits for the bitmap are just 138 bytes from index 0.


Neat, on to the next header!

This follows the format below:
![img](http://i.imgur.com/zfAZchy.png)



biSize describes the size of the bitmap structure. biWidth is how many pixels wide the image is. And biHeight is the height in pixels.

biPlanes is always 1. It says "number of planes for the target device," and I'm curious what exactly this was meant for. Stereovision? Animated gifs? Who knows.

biBitCount is the number of bits per pixel. So, a purely black and white image would have one bit, and for any other value we'd have 2^n possible values. `file` tells us to expect 24 bits.

biCompression specifies what compression scheme is used. We aren't using one, so we should get the constant for no compression here. Aparently, the BMP format can encode JPEG or PNG formated images as well, which I didn't know and haven't really encountered to my knowledge.

biSizeImage is the size of the image data in the file in bytes. In our case, this will actually be 0 because it's redundant.

biXPelsPerMeter and biYPelsPerMeter are units like DPI, in that they specify how the image should be scaled for printing or a monitor of a certain size.

biClrUsed is the number of colors in the bitmap, and biClrImportant is the number of colors required to display the image. I'm not quite clear on what the difference is or why there are two fields.

In [42]:
biSize, biWidth, biHeight, biPlanes, biBitCount,\
    biCompression, biSizeImage, biXPelsPerMeter, \
    biYPelPerMeter, biClrUsed, biClrImportant = struct.unpack("=IllHHIIllII", bmp[14:14+40])
biSize

124